In [1]:
from lifelines.datasets import load_rossi
from synthcity.plugins.core.dataloader import SurvivalAnalysisDataLoader
from synthcity.plugins import Plugins

X = load_rossi()
data = SurvivalAnalysisDataLoader(
    X,
    target_column="arrest",
    time_to_event_column="week",
)



[KeOps] Warning : Cuda libraries were not detected on the system or could not be loaded ; using cpu only mode


In [3]:
syn_model = Plugins().get("survival_gan")
syn_model

[2024-12-13T18:04:14.317467-0500][17828][CRITICAL] module disabled: C:\Users\tkvkh\AppData\Local\Programs\Python\Python311\Lib\site-packages\synthcity\plugins\generic\plugin_goggle.py


In [6]:
import pandas as pd

train = pd.read_csv("data/train.csv")

RMV = ["ID", "efs", "efs_time", "y"]
FEATURES = [c for c in train.columns if not c in RMV]
print(f"Total features: {len(FEATURES)} - {FEATURES}")

CATS = []
for c in FEATURES:
    if train[c].dtype=="object":
        CATS.append(c)
        train[c] = train[c].fillna("NAN")
print(f"In these features, there are {len(CATS)} CATEGORICAL FEATURES: {CATS}")

print("We LABEL ENCODE the CATEGORICAL FEATURES: ",end="")
for c in FEATURES:

    # LABEL ENCODE CATEGORICAL AND CONVERT TO INT32 CATEGORY
    if c in CATS:
        print(f"{c}, ",end="")
        train[c],_ = train[c].factorize()
        train[c] -= train[c].min()
        train[c] = train[c].astype("int32")
        train[c] = train[c].astype("category")
        
    # REDUCE PRECISION OF NUMERICAL TO 32BIT TO SAVE MEMORY
    else:
        train[c] = train[c].fillna(-1)
        if train[c].dtype=="float64":
            train[c] = train[c].astype("float32")
        if train[c].dtype=="int64":
            train[c] = train[c].astype("int32")

data = SurvivalAnalysisDataLoader(
    train,
    target_column="efs",
    time_to_event_column="efs_time",
)

Total features: 57 - ['dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'hla_match_c_high', 'hla_high_res_8', 'tbi_status', 'arrhythmia', 'hla_low_res_6', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe', 'prim_disease_hct', 'hla_high_res_6', 'cmv_status', 'hla_high_res_10', 'hla_match_dqb1_high', 'tce_imm_match', 'hla_nmdp_6', 'hla_match_c_low', 'rituximab', 'hla_match_drb1_low', 'hla_match_dqb1_low', 'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity', 'year_hct', 'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hla_match_a_high', 'hepatic_severe', 'donor_age', 'prior_tumor', 'hla_match_b_low', 'peptic_ulcer', 'age_at_hct', 'hla_match_a_low', 'gvhd_proph', 'rheum_issue', 'sex_match', 'hla_match_b_high', 'race_group', 'comorbidity_score', 'karnofsky_score', 'hepatic_mild', 'tce_div_match', 'donor_related', 'melphalan_dose', 'hla_low_res_8', 'cardiac', 'hla_match_drb1_high', 'pulm_moderate', 'hla_low_res_10']
In these features, there are 35 CATEGORICA

In [7]:
syn_model.fit(data)

[2024-12-13T18:15:44.008916-0500][17828][CRITICAL] module disabled: C:\Users\tkvkh\AppData\Local\Programs\Python\Python311\Lib\site-packages\synthcity\plugins\generic\plugin_goggle.py
  0%|                                                                            | 12/10000 [04:44<65:45:22, 23.70s/it]


KeyboardInterrupt: 